In [ ]:
Model only accept vectors as input

we are required to convert the text string to numbers and vectorize
them 

Two very common vectorizing:
    One-hot encoding
    Unique number encoding

In [ ]:
# One-Hot Encoding
create a zero vector of size of array equal to the vocab and add
1 to the indices corresponding to the word

the approach is inefficient, The type of vector is sparse(most indices are zero)

In [ ]:
# Unique Number Encoding

encode each word with a unique number

Drawback:
    the integer encoding doesn't capture relationships
    the lack of relationship makes it hard to interpret

In [ ]:
#Word Embedding
 a dense vector of floating points values with length of the 
    vector provided as a parameter

no manual assignment of values, encoding are trainable params
    that are learned furing model training

embeddings offers an efficient and dense vector to 
    represent the relationship between words


# Word Embeddings for Sentiment Analysis

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
print(tf.__version__)

2.3.0


In [4]:
#We will get both metadata and labels to them
data, info = tfds.load("imdb_reviews", with_info = True, as_supervised = True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling imdb_reviews-train.tfrecord...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling imdb_reviews-test.tfrecord...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling imdb_reviews-unsupervised.tfrecord...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to C:\Users\Denylson\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [7]:
train_data, test_data = data['train'], data['test']

train_sentences = []
test_sentences = []

train_labels = []
test_labels = []

In [8]:
for sent, label in train_data:
    train_sentences.append(str(sent.numpy().decode('utf8')))
    train_labels.append(label.numpy())

for sent, label in test_data:
    test_sentences.append(str(sent.numpy().decode('utf8')))
    test_labels.append(label.numpy())

In [9]:
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [10]:
vocab_size = 1000
emb_size = 32
max_length = 150
trunc_type = 'post'
oov_tok = "<oov>"

In [13]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

train_seqs = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_seqs, maxlen = max_length, truncating = trunc_type)

test_seqs = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_seqs, maxlen = max_length, truncating = trunc_type)

In [14]:
reverse_word_index = dict([(value, key)  for (key, value) in word_index.items()])

def decode_review(text):
    return ''.join([reverse_word_index.get(i, '?') for i in text])

print(train_sentences[1])
print(train_padded[1])
print(decode_review(train_padded[1]))

I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no explanation of what was causing them or why. I admit, I may have missed part of the film, but i watched the majority of it and everything just seemed to happen of its own accord without any real concern for anything else. I cant recommend this film at all.
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0  11  26  75 571   6 805   1 313 106  19  12   7 629 686   6   4
   1   5 181 584  64   1 110   1   3   1  21   2   1   3 258  41   1   4
 174 188  21  12   1  11   1   1  86   2  20  14   1   2 112 940  14   1
   1 548   3 355 

# Defining a Nueral Network with Embedding layer

In [17]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, emb_size, input_length = max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 32)           32000     
_________________________________________________________________
flatten_1 (Flatten)          (None, 4800)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 28806     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 60,813
Trainable params: 60,813
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Training

num_epochs = 10

model.fit(
    train_padded,
    train_labels,
    epochs = num_epochs,
    validation_data = (test_padded, test_labels)
)

Epoch 1/10
782/782 [==============================] - 1s 1ms/step - loss: 0.5148 - accuracy: 0.7268 - val_loss: 0.3976 - val_accuracy: 0.8201
Epoch 2/10
782/782 [==============================] - 1s 1ms/step - loss: 0.3384 - accuracy: 0.8567 - val_loss: 0.4132 - val_accuracy: 0.8143
Epoch 3/10
782/782 [==============================] - 1s 1ms/step - loss: 0.2384 - accuracy: 0.9101 - val_loss: 0.4776 - val_accuracy: 0.7943
Epoch 4/10
782/782 [==============================] - 1s 1ms/step - loss: 0.1358 - accuracy: 0.9571 - val_loss: 0.6153 - val_accuracy: 0.7876
Epoch 5/10
782/782 [==============================] - 1s 1ms/step - loss: 0.0654 - accuracy: 0.9850 - val_loss: 0.7463 - val_accuracy: 0.7820
Epoch 6/10
782/782 [==============================] - 1s 1ms/step - loss: 0.0301 - accuracy: 0.9952 - val_loss: 0.8702 - val_accuracy: 0.7827
Epoch 7/10
782/782 [==============================] - 1s 1ms/step - loss: 0.0140 - accuracy: 0.9978 - val_loss: 0.9742 - val_accuracy: 0.7816
Epoch 

# Deriving weight from  Embedding layer

In [19]:
l1 = model.layers[0]
weights = l1.get_weights()[0]
print(weights.shape)
print(weights)

(1000, 32)
[[ 0.03659494  0.00098447 -0.00871876 ...  0.01547277 -0.01957596
  -0.02067693]
 [ 0.03770637 -0.00848457 -0.02598854 ... -0.04054574 -0.00193059
  -0.03420943]
 [ 0.03052573  0.04776368  0.02018204 ...  0.04097027  0.01607944
   0.00581276]
 ...
 [ 0.15759665 -0.20130087  0.09648244 ...  0.05359149  0.00288965
  -0.13382825]
 [ 0.11952089 -0.04653597 -0.15234673 ...  0.10648727 -0.15857537
   0.0440921 ]
 [ 0.02776677 -0.20963357 -0.1583334  ...  0.10204659 -0.15585104
  -0.0685412 ]]


# Part 2 (Projecting embeddings on tensorflow projector)

In [21]:
import io

vectors = io.open('vectors.tsv', 'w', encoding = 'utf-8')

meta = io.open('meta.tsv', 'w', encoding= 'utf-8')

#write each word and its correspoding metadata
for index in range(1, vocab_size):
    word = reverse_word_index[index]
    embeddings = weights[index]
    meta.write(word + '\n')
    vectors.write('\t'.join([str(x) for x in embeddings]) + '\n')

vectors.close()
meta.close()

# Classifying News headlines

In [ ]:
import pandas as pd

data = pd.read_json('./xl.json')
data.head()

In [ ]:
headlines = list(data['headline'])
labels = list(data['is_sarcastic'])

In [ ]:
#Setting Parameters

vocab_size = 10000
max_length = 120
emb_size = 32
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<oov>'

training_size = 20000

In [ ]:
#splitting data

training_sentences = headlines[0:training_size]
testing_sentences = headlines[training_size:]

training_labels = labels[0:training_size]
testing_labels = labels[training:]

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_senteces)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen = max_length, padding = padding_type, truncating = trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen = max_length, padding = padding_type, truncating = trunc_type)


In [ ]:
#convert to np.array to work with Tensorflow 2.x
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, emb_size, input_length = max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
    
])

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['loss', 'accuracy'])

In [ ]:
history = model.fit(
    training_padded, 
    training_labels,
    epochs = num_epochs,
    validation_data = (testing_padded, testing_labels)
    verbose = 2
)

In [ ]:
# Visualization of loss and accuracy
import matplotlib.pyplot as plt

def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_' + string])
    plt.legend([string, 'val_' + string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.show()

plot_graphs(history, 'accuracy')
plot_graphs(history, 'loss')    

In [ ]:
#Classifying a new sentence

sentence = ['the baby boy fears spiders in the garden in the garden might be real', 'game of thrones season finale showing this sunday night']

#prepare the senquences of the sentences in question
sequences = tokenizer.text_to_sequences(sentence)
padded_seqs = pad_sequences(sequences, maxlen = max_length, padding = padding_type, truncating = trunc_type)

print(model.predict(padded_seqs))

# Text Classification Challenge

In [23]:
data, info = tfds.load('imdb_reviews', with_info = True, as_supervised = True)


In [25]:
train_data, test_data = data['train'], data['test']

train_senteces = []
test_senteces = []

train_labels = []
test_labels = []

In [27]:
for sentence, label in train_data:
    train_sentences.append(str(sentence.numpy().decode('utf8')))
    train_labels.append(label.numpy())
for sentence, label in test_data:
    test_sentences.append(str(sentence.numpy().decode('utf8')))
    test_labels.append(label.numpy())

In [28]:
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [29]:
#parameters

vocab_size = 10000
emb_size = 50
max_length = 100
trunc_type = 'post'
oov_tok = '<oov>'

In [31]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token = '<oov>')
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

#training sequences and labels

train_seq = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_seq, maxlen = max_length, truncating = trunc_type)

test_seq = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_seq, maxlen = max_length)

In [41]:
num_epochs = 10
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, emb_size, input_length = max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [43]:
model.fit(
    train_padded,
    train_labels,
    epochs = num_epochs,
    validation_data = (test_padded, test_labels)
)

ValueError: Data cardinality is ambiguous:
  x sizes: 50001
  y sizes: 25000
Please provide data which shares the same first dimension.

In [37]:
import matplotlib.pyplot as plt

def plot_metric(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_' + metric])
    plt.legend([string, 'val_'+ metric])
    plt.xlabel("epochs")
    plt.ylabel(metric)
    plt.show()
    
plot_metric(history, 'accuracy')
plot_metric(history, 'loss')

AttributeError: 'NoneType' object has no attribute 'history'